# NOTE
This is an accompanying notebook for the post [Wealth, Games and Monte Carlo](https://theperceptron.substack.com/p/monte-carlo).
Check it out on [The Perceptron](https://theperceptron.substack.com/welcome).

# Imports

In [ ]:
import numpy as np
from numpy import linalg as la
import pandas as pd
import matplotlib.pyplot as  plt
from matplotlib import image
import seaborn as sns
import imageio
import random
import datetime
import os

sns.set()
%matplotlib inline

# Estimation of π

In [ ]:
# Draw a square with a quadrant of radius within it
figure, axes = plt.subplots()
circle = plt.Circle((0,0), 1, fill = False, color='k')
axes.set_aspect(1)
axes.add_patch(circle)
plt.show()

In [ ]:
filenames = []
size = [1, 10, 100, 1000, 10000, 100000]

for i, n in enumerate(size):
    print(f'Calculating for n = {n}...')
    points = np.random.rand(n, 2)
    col = list(np.where(la.norm(points, axis=1) <= 1, 'r', 'b'))
    
    figure, axes = plt.subplots()
    figure.set_size_inches(4, 4)
    axes.set_aspect(1)
    axes.set_ylim(ymin=0)
    axes.set_xlim(xmin=0)
    
    circle = plt.Circle((0,0), 1, fill = False, color='k')
    axes.add_patch(circle)
    plt.scatter(points[:, 0], points[:, 1], c=col, marker='.', s=0.5)
    
    pi = 4 * (np.sum(np.where(la.norm(points, axis=1) <= 1, 1, 0)))/n
    
    plt.title(f'n = {n}, π ≈ {pi:.4f}')
    
    filename = f'{i}.png'
    filenames.append(filename)
    plt.savefig(filename, dpi=100)
    plt.close()

In [ ]:
# Build gif
with imageio.get_writer('pi.gif', mode='I', duration = 1) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Remove files
for filename in set(filenames):
    os.remove(filename)

<img src="pi.gif" width="300" align="center">

# Dice game

In [ ]:
n = 10000
dice = [1, 2, 3, 4, 5, 6]

In [ ]:
# When you choose even
final_reward_even = 0
even_rewards = []

for i in range(n):
    roll = random.choice(dice)
    if roll % 2:
        final_reward_even -= roll
    else:
        final_reward_even += roll
    even_rewards.append(final_reward_even)

plt.plot(even_rewards)
plt.title(f'Final reward when choosing even: {final_reward_even}')
plt.show()

In [ ]:
# When you choose odd
final_reward_odd = 0
odd_rewards = []

for i in range(n):
    roll = random.choice(dice)
    if roll % 2:
        final_reward_odd += roll
    else:
        final_reward_odd -= roll
    odd_rewards.append(final_reward_odd)

plt.plot(odd_rewards)
plt.title(f'Final reward when choosing odd: {final_reward_odd}')
plt.show()

# Gaussian Integral

In [ ]:
left_limit = -10
right_limit = 10

x = np.arange(left_limit, right_limit, 0.001)
y = np.e**(-(x)**2)

plt.plot(x, y, 'k')

plt.show()

In [ ]:
filenames = []
size = [1, 10, 100, 1000, 10000, 100000]

for i, n in enumerate(size):
    print(f'Calculating for n = {n}...')
    points_x = np.random.uniform(low=-10, high=10, size=n)
    points_y = np.random.uniform(low=0, high=1, size=n)
    col = list(np.where(points_y <= np.e**(-(points_x)**2), 'r', 'b'))
    
    figure, axes = plt.subplots()
    figure.set_size_inches(10, 4)
    
    left_limit = -10
    right_limit = 10

    x = np.arange(left_limit, right_limit, 0.001)
    y = np.e**(-(x)**2)

    plt.plot(x, y, 'k')
    
    plt.scatter(points_x, points_y, c=col, marker='.', s=0.5)
    
    val = (right_limit - left_limit)*(np.sum(np.where(points_y <= np.e**(-(points_x)**2), 1, 0)))/n
    
    plt.title(f'n = {n}, val ≈ {val:.4f}, real val = {np.sqrt(np.pi):.4f}')
    
    filename = f'{i}.png'
    filenames.append(filename)
    plt.savefig(filename, dpi=100)
    plt.close()

In [ ]:
# Build gif
with imageio.get_writer('gauss.gif', mode='I', duration = 1) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Remove files
for filename in set(filenames):
    os.remove(filename)

<img src="gauss.gif" width="600" align="center">

# Portfolio Optimization

Implemented from: https://www.kaggle.com/ethanhunt1080/effective-portfolio-using-monte-carlo-simulation

Dataset: https://www.kaggle.com/rohanrao/nifty50-stock-market-data/metadata

In [ ]:
# Reading the dataset using pandas and storing it in a dictionary

raw_data = {}
raw_data['RELIANCE'] = pd.read_csv('../input/nifty50-stock-market-data/RELIANCE.csv')
raw_data['HDFCBank'] = pd.read_csv('../input/nifty50-stock-market-data/HDFCBANK.csv')
raw_data['INFY'] = pd.read_csv('../input/nifty50-stock-market-data/INFY.csv')
raw_data['TCS'] = pd.read_csv('../input/nifty50-stock-market-data/TCS.csv')
raw_data['HINDUNILVR'] = pd.read_csv('../input/nifty50-stock-market-data/HINDUNILVR.csv')

In [ ]:
# Checking the dataset

raw_data['RELIANCE']

In [ ]:
# Extracting the data fo last 365 days into a dataframe called datas

datas = {}

for name, data in raw_data.items():
    datas[name] = raw_data[name].iloc[-365:].reset_index()

datas['RELIANCE']

In [ ]:
#  Extracting the closing price of the coins in a dataframe called CP_dict

cp_dict = {}

for name, data in datas.items():
    cp_dict[name] = datas[name]['Close']
    
cp_df = pd.DataFrame(cp_dict)

cp_df

In [ ]:
# The log daily returns and removing the first empty column of each stock
daily_return = np.log(cp_df.pct_change() + 1).dropna()

# Calculating the mean of those daily return
daily_return_mean = np.array(daily_return.mean())

# Assigning weights. Since there are five stocks, each will have a weight of 0.2 (1/5)
weights = np.array([0.2,0.2,0.2,0.2,0.2])

In [ ]:
# Calculating Portfolio Return
port_return = np.sum(weights * daily_return_mean) * 365

print(f'The Annual Return of Portfolio is {port_return * 100:.2f}')

In [ ]:
# Calculating Portfolio Volatility
cov = daily_return.cov()
port_vol = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(365)

print(f'The Annual Volatility of Portfolio is {port_vol * 100:.2f}%')

In [ ]:
equal_proportion = [port_return, port_vol]

In [ ]:
# The number of Portfolios to be generated
num_portfolio = 30000

# Creating an empty list for storing returns, volatility, sharpe_ratio (return/volatility) and weightage of each stock in portfolio
results = np.zeros((3 + len(daily_return.columns), num_portfolio))

# Monte Carlo Simulation
for i in range(num_portfolio):
    
    # Declaring random weights
    weight = np.random.rand(len(daily_return.columns))
    # Ensuring the sum of all weight to be equal to 1
    weight = weight/np.sum(weight)

    # Annual Return
    p_annual_return = np.sum(weight * daily_return_mean) * 365
    # Annual Volatility
    p_annual_volatility = np.sqrt(np.dot(weight.T, np.dot(cov, weight))) * np.sqrt(365)
    
    # Storing the values in results list
    results[0, i] = p_annual_return
    results[1, i] = p_annual_volatility
    results[2, i] = results[0,i] / results[1,i]

    for j in range(len(weight)):
        results[j+3,i] = weight[j]
        
        
# Making a dataframe for results list of all generated Portfolio
cols = ['ann_ret', 'ann_vol', 'sharpe_ratio']
for num in range(len(list(daily_return.columns))):
    cols.append(list(daily_return.columns)[num])

result_df = pd.DataFrame(results.T, columns=cols)

In [ ]:
# Visulising the result dataframe
result_df

In [ ]:
# Portfolio 1 - Sharpe ratio is the highest (Return/Volatility)
max_sharpe_ratio = result_df.iloc[result_df['sharpe_ratio'].idxmax()]


# Portfolio 2 - Volatility is the lowest
volatility_lowest = result_df.iloc[result_df['ann_vol'].idxmin()]

In [ ]:
# Plotting the simulation
plt.figure(figsize=(15,8))
plt.scatter(result_df['ann_vol'],result_df['ann_ret'],c =result_df['sharpe_ratio'], cmap='rocket_r', label='_nolegend_')
plt.colorbar()

# Red - Portfolio 1
plt.scatter(max_sharpe_ratio[1], max_sharpe_ratio[0], marker='*',color='red', s=300)

# Green - Portfolio 2
plt.scatter(volatility_lowest[1], volatility_lowest[0], marker='*',color='green', s=300)

# Blue - Portfoliio 3
plt.scatter(equal_proportion[1], equal_proportion[0], marker='*',color='blue', s=300)

plt.xlabel('Volatility',fontsize = 15)
plt.ylabel('Returns',fontsize = 15)
plt.legend(['Max. Sharpe Ratio', 'Min. Volatility', 'Equal Proportion'])
plt.show()

In [ ]:
##### print('The Portfolio with maximum return (Sharpe Ratio) is:')
print('All values in percentage')
print(round(max_sharpe_ratio * 100, 2))

In [ ]:
print('The Portfolio with least Volatility is:')
print('All values in percentage')
print(round(volatility_lowest * 100, 2))